In [31]:
# Import Libraries

import os
import glob
import copy
import time
import random

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from collections import namedtuple
import SimpleITK as sitk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [32]:
# DATA EXPLORE 
archive_path = r"C:\Users\azizd\python\pytorch\data\bitirme_projesi\archive"

In [33]:
# 1- annotations.csv
df_annotations = pd.read_csv(archive_path + r"\annotations.csv")
print("Shape: ",df_annotations.shape)
df_annotations.head()

Shape:  (1186, 5)


,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [34]:
# 2- candidates_V2.csv

df_candidates = pd.read_csv(archive_path + r"\candidates_V2\candidates_V2.csv")
print("Shape: ", df_candidates.shape)
df_candidates.head()

Shape:  (754975, 5)


,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


In [35]:
# Note: There can be multiple annotations and candidates in a single CT scan.

print(f'Total annotations: {df_annotations.shape[0]}, Unique CT scans: {len(df_annotations.seriesuid.unique())}')
print(f'Total candidates: {df_candidates.shape[0]}, Unique CT scans: {len(df_candidates.seriesuid.unique())}')

Total annotations: 1186, Unique CT scans: 601
Total candidates: 754975, Unique CT scans: 888


In [36]:
"""
We will first group all annotations that are part of the same CT scan (i.e. have the same seriesuid).

This will allow us to easily access the centers and respective diameters of all nodules in a particular CT scan.
"""

# The `diameters` dict will have each `seriesuid` as a key.
# The value will be a list of all center coorinates in the CT scan with that `seriesuid`
diameters = {}

# Loop through every annotation
for _, row in df_annotations.iterrows():
    
    # Create a tuple to represent the center
    center_xyz = (row.coordX, row.coordY, row.coordZ)
    
    # Append the center to the corresponding `seriesuid`
    diameters.setdefault(row.seriesuid, []).append(
        (center_xyz, row.diameter_mm)
    )

In [37]:
%%time

# Using a namedtuple makes it easy to access values in a tuple
# using indexes or field names
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    ['is_nodule', 'diameter_mm', 'series_uid', 'center_xyz']
)

# A list to store all candidates in the dataset
candidates = []

for _, row in df_candidates.iterrows():
    
    # Create a tuple to represent the candidate center
    # We suffix the name with `_xyz` to make it clear that we're using the
    # patient coordinate system: http://dicomiseasy.blogspot.com/2013/06/getting-oriented-using-image-plane.html
    candidate_center_xyz = (row.coordX, row.coordY, row.coordZ)

    # We begin by assuming the candidate doesn't have a corresponding annotation.
    # If this is the case, then the candidate will have a diameter of 0.
    candidate_diameter = 0.0
    
    # We then fetch the diameters of the CT scan we're looking at currently,
    # and loop over them to find a match for the candidate
    for annotation in diameters.get(row.seriesuid, []):
        
        # Extract the center and diameter of the annotation from the tuple
        annotation_center_xyz, annotation_diameter = annotation
        
        # For each of the coordinates - X, Y and Z, we check if
        # the candidate and the annotation are "close by"
        # (remember the really long and complicated sentence above?)
        
        # Since we've stored coordinates as tuples, we can index into them
        for i in range(3):
            
            # Find the absolute difference between the two coordinates
            delta = abs(candidate_center_xyz[i] - annotation_center_xyz[i])
            
            # If the coorindate of the candidate is more than half the radius away,
            # we don't consider it the same nodule as the annotation we're currently looking at
            if delta > annotation_diameter / 4:
                    break
            
        # The `else` block of a for loop in Python executes if the loop ends "naturally"
        # i.e. if it terminates because all iterations are complete, and not by a break statement
        # So if we go into this else block, then all 3 coordinates are within half the radius,
        # and we can consider the candidate and the annotation as the same nodule
        else:
            candidate_diameter = annotation_diameter
            
            # We don't need to look at any other remaining annotations,
            # because we've already found a match!
            break
            
            
    
    candidates.append(CandidateInfoTuple(
        bool(row['class']),
        candidate_diameter,
        row.seriesuid,
        candidate_center_xyz
    ))

CPU times: total: 33.8 s
Wall time: 34.5 s


In [38]:
candidates.sort(reverse=True)
candidates[0]

CandidateInfoTuple(is_nodule=True, diameter_mm=32.27003025, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886', center_xyz=(66.58022805, 82.56931698, -110.5421104))

In [39]:
# Handling the missing data

#  missing.txt: https://www.kaggle.com/mashruravi/luna16missingcandidates

with open(archive_path + r"\missing.txt", 'r') as f:
    missing_uids = {uid.split('\n')[0] for uid in f}
    
len(missing_uids)

443

In [40]:
# Remove missing cadidates

candidates_clean = list(filter(lambda x: x.series_uid not in missing_uids, candidates))

print(f'All candidates in dataset: {len(candidates)}')
print(f'Candidates with CT scan  : {len(candidates_clean)}')

All candidates in dataset: 754975
Candidates with CT scan  : 377138


In [41]:
candidate = candidates_clean[0]
candidate

CandidateInfoTuple(is_nodule=True, diameter_mm=32.27003025, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886', center_xyz=(66.58022805, 82.56931698, -110.5421104))

In [42]:
# Look for the file `<series_uid>.mhd` inside the `subset` folders
filepaths = glob.glob(archive_path + f"/subset*/*/{candidate.series_uid}.mhd")

# We removed all candidates that don't have corresponding CT scan files
# This line is another fail-safe to know when a CT scan doesn't exist
assert len(filepaths) != 0, f'CT scan with seriesuid {candidate.series_uid} not found!'

filepaths

['C:\\Users\\azizd\\python\\pytorch\\data\\bitirme_projesi\\archive\\subset1\\subset1\\1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886.mhd']

In [43]:
mhd_file_path = filepaths[0]

mhd_file_path

'C:\\Users\\azizd\\python\\pytorch\\data\\bitirme_projesi\\archive\\subset1\\subset1\\1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886.mhd'

In [44]:
mhd_file = sitk.ReadImage(mhd_file_path)

In [45]:
ct_scan = np.array(sitk.GetArrayFromImage(mhd_file), dtype=np.float32)

In [87]:
ct_scan.clip(-1000, 1000, ct_scan)[1]

array([[-1000., -1000., -1000., ..., -1000., -1000., -1000.],
       [-1000., -1000., -1000., ..., -1000., -1000., -1000.],
       [-1000., -1000., -1000., ..., -1000., -1000., -1000.],
       ...,
       [-1000., -1000., -1000., ..., -1000., -1000., -1000.],
       [-1000., -1000., -1000., ..., -1000., -1000., -1000.],
       [-1000., -1000., -1000., ..., -1000., -1000., -1000.]],
      dtype=float32)

In [47]:
origin_xyz = mhd_file.GetOrigin()
voxel_size_xyz = mhd_file.GetSpacing()
direction_matrix = np.array(mhd_file.GetDirection()).reshape(3, 3)

In [48]:
origin_xyz_np = np.array(origin_xyz)
voxel_size_xyz_np = np.array(voxel_size_xyz)

In [49]:
# Convert the coordinates of the center of the candidate
# from the patient coordinate system to column, row, index
cri = ((center_xyz - origin_xyz_np) @ np.linalg.inv(direction_matrix)) / voxel_size_xyz_np

# Since we'll be using column, row and index values to index into arrays,
# we round them to the nearest integer.
cri = np.round(cri)

# Going forward, we'll need the scan to be in the order index, row, column
irc = (int(cri[2]), int(cri[1]), int(cri[0]))

In [50]:
ct_scan.shape

(123, 512, 512)

In [51]:
dims_irc = (10, 18, 18)

In [52]:
# We will create three slices - one for each direction - to use to extract
# a region of interest from the CT scan
slice_list = []

for axis, center_val in enumerate(irc):
    
    # Get start and end index for the dimension so that the
    # nodule center is at the center of the 3d array we extract
    start_index = int(round(center_val - dims_irc[axis]/2))
    end_index = int(start_index + dims_irc[axis])

    # Adjust the indexes if the start_index is out of the CT scan array
    if start_index < 0:
        start_index = 0
        end_index = int(dims_irc[axis])
    
    # Do the same check for the end_index
    if end_index > ct_scan.shape[axis]:
        end_index = ct_scan.shape[axis]
        start_index = int(ct_scan.shape[axis] - dims_irc[axis])
        
    slice_list.append(slice(start_index, end_index))
    
tuple(slice_list)

(slice(68, 78, None), slice(288, 306, None), slice(223, 241, None))

In [53]:
ct_scan_chunk = ct_scan[tuple(slice_list)]
ct_scan_chunk.shape

(10, 18, 18)

In [54]:
# Create a tensor from the NumPy array of the CT scan chunk
ct_scan_chunk_tensor = torch.from_numpy(ct_scan_chunk)

# convert it to a tensor of float32
ct_scan_chunk_tensor = ct_scan_chunk_tensor.to(torch.float32)
    
# Add an extra dimension to represent a single channel in the 3d image
ct_scan_chunk_tensor = ct_scan_chunk_tensor.unsqueeze(0)

ct_scan_chunk_tensor.shape

torch.Size([1, 10, 18, 18])

In [55]:
candidate.is_nodule

True

In [56]:
torch.tensor([
    not candidate.is_nodule,
    candidate.is_nodule,
], dtype=torch.long)

tensor([0, 1])

In [57]:
!pip install diskcache cassandra-driver

In [58]:
# The code in this cell is from the Deep Learning with PyTorch book's GitHub repository
# https://github.com/deep-learning-with-pytorch/dlwpt-code/blob/master/util/disk.py

# The imports have slightly been modified to make the code work


import gzip

from cassandra.cqltypes import BytesType
from diskcache import FanoutCache, Disk, core
from diskcache.core import io, MODE_BINARY
from io import BytesIO

class GzipDisk(Disk):
    def store(self, value, read, key=None):
        """
        Override from base class diskcache.Disk.

        Chunking is due to needing to work on pythons < 2.7.13:
        - Issue #27130: In the "zlib" module, fix handling of large buffers
          (typically 2 or 4 GiB).  Previously, inputs were limited to 2 GiB, and
          compression and decompression operations did not properly handle results of
          2 or 4 GiB.

        :param value: value to convert
        :param bool read: True when value is file-like object
        :return: (size, mode, filename, value) tuple for Cache table
        """
        # pylint: disable=unidiomatic-typecheck
        if type(value) is BytesType:
            if read:
                value = value.read()
                read = False

            str_io = BytesIO()
            gz_file = gzip.GzipFile(mode='wb', compresslevel=1, fileobj=str_io)

            for offset in range(0, len(value), 2**30):
                gz_file.write(value[offset:offset+2**30])
            gz_file.close()

            value = str_io.getvalue()

        return super(GzipDisk, self).store(value, read)


    def fetch(self, mode, filename, value, read):
        """
        Override from base class diskcache.Disk.

        Chunking is due to needing to work on pythons < 2.7.13:
        - Issue #27130: In the "zlib" module, fix handling of large buffers
          (typically 2 or 4 GiB).  Previously, inputs were limited to 2 GiB, and
          compression and decompression operations did not properly handle results of
          2 or 4 GiB.

        :param int mode: value mode raw, binary, text, or pickle
        :param str filename: filename of corresponding value
        :param value: database value
        :param bool read: when True, return an open file handle
        :return: corresponding Python value
        """
        value = super(GzipDisk, self).fetch(mode, filename, value, read)

        if mode == MODE_BINARY:
            str_io = BytesIO(value)
            gz_file = gzip.GzipFile(mode='rb', fileobj=str_io)
            read_csio = BytesIO()

            while True:
                uncompressed_data = gz_file.read(2**30)
                if uncompressed_data:
                    read_csio.write(uncompressed_data)
                else:
                    break

            value = read_csio.getvalue()

        return value

def getCache(scope_str):
    return FanoutCache('data-unversioned/cache/' + scope_str,
                       disk=GzipDisk,
                       shards=64,
                       timeout=1,
                       size_limit=3e11,
                       )

raw_cache = getCache('ct_scan_raw')

@raw_cache.memoize(typed=True)
def getCtScanChunk(series_uid, center_xyz, dims_irc):
        filepaths = glob.glob(archive_path + f"/subset*/*/{series_uid}.mhd")
        #filepaths = glob.glob(f'/kaggle/input/luna16/subset*/*/{series_uid}.mhd')
        assert len(filepaths) != 0, f'CT scan with seriesuid {series_uid} not found!'
        mhd_file_path = filepaths[0]
        
        mhd_file = sitk.ReadImage(mhd_file_path)
        ct_scan = np.array(sitk.GetArrayFromImage(mhd_file), dtype=np.float32)
        ct_scan.clip(-1000, 1000, ct_scan)
        
        origin_xyz = mhd_file.GetOrigin()
        voxel_size_xyz = mhd_file.GetSpacing()
        direction_matrix = np.array(mhd_file.GetDirection()).reshape(3, 3)
        
        origin_xyz_np = np.array(origin_xyz)
        voxel_size_xyz_np = np.array(voxel_size_xyz)
        
        cri = ((center_xyz - origin_xyz_np) @ np.linalg.inv(direction_matrix)) / voxel_size_xyz_np
        cri = np.round(cri)
        irc = (int(cri[2]), int(cri[1]), int(cri[0]))
        
        slice_list = []
        for axis, center_val in enumerate(irc):
            
            start_index = int(round(center_val - dims_irc[axis]/2))
            end_index = int(start_index + dims_irc[axis])
            
            if start_index < 0:
                start_index = 0
                end_index = int(dims_irc[axis])
                
            if end_index > ct_scan.shape[axis]:
                end_index = ct_scan.shape[axis]
                start_index = int(ct_scan.shape[axis] - dims_irc[axis])

            slice_list.append(slice(start_index, end_index))
            
        ct_scan_chunk = ct_scan[tuple(slice_list)]
        
        return ct_scan_chunk

In [59]:
class LunaDataset(Dataset):
    
    def __init__(self, is_validation_set=False, validation_stride=0):
        '''Create a PyTorch dataset for the CT scans
        
        If `is_validation_set` is `True` then every `validation_stride` item is kept.
        Otherwise, every `validation_stride` item is deleted
        '''
        
        # Make a copy of all the candidates.
        # Pick every 350th candidate so that we have about 1k candidates in the dataset
        # It takes agonizingly long to load more data!
        positives = [c for c in candidates_clean if c.is_nodule]
        negatives = [c for c in candidates_clean if not c.is_nodule]
        
        # Undersample negatives to match positives
        negatives = negatives[:len(positives)]
        self.candidates = positives + negatives
        random.shuffle(self.candidates)

        
        # If this is the validation set, keep every `validation_stride` item
        if is_validation_set:
            self.candidates = self.candidates[::validation_stride]
        
        # If this is the training set, delete every `validation_stride` item
        else:
            del self.candidates[::validation_stride]
            
    def __len__(self):
        '''Returns the number of items in the dataset'''
        return len(self.candidates)
    
    def __getitem__(self, i):
        '''Get the `i`the item in the dataset'''
        
        # Get the `i`th candidate
        candidate = self.candidates[i]
        
        # We want to resize each CT scan to the following dimensions
        dims_irc = (10, 18, 18)
        
        # Use the utility function to fetch the CT scan
        ct_scan_np = getCtScanChunk(candidate.series_uid, candidate.center_xyz, dims_irc)
        
        # Convert the CT scan to a tensor
        ct_scan_tensor = torch.from_numpy(ct_scan_np).to(torch.float32).unsqueeze(0)
        
        # Convert the target to a tensor
        label_tensor = torch.tensor(1 if candidate.is_nodule else 0, dtype=torch.long)

        
        return ct_scan_tensor, label_tensor

In [60]:
VALIDATION_STRIDE=10
BS=16

train_ds = LunaDataset(is_validation_set=False, validation_stride=VALIDATION_STRIDE)
val_ds = LunaDataset(is_validation_set=True, validation_stride=VALIDATION_STRIDE)

train_dl = DataLoader(train_ds, batch_size=BS, num_workers=0)
val_dl = DataLoader(val_ds, batch_size=BS, num_workers=0)

In [81]:
def train_loop(model, dataloader, criterion, optimizer, ds_size):
    '''Train the model for one epoch'''
    
    # Put the model in training mode to activate dropout
    model.train()
    
    # Keep a track of the loss and correct predictions for the epoch
    running_loss = 0.0
    running_corrects = 0
    
    # Track the total number of positives and true positives
    running_pos = 0
    running_pos_correct = 0
    
    # Track the total number of negatives and true negatives
    running_neg = 0
    running_neg_correct = 0
    
    for inputs, labels in tqdm(dataloader):
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # --- Standard PyTorch training process ---
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        # -----------------------------------------
        
        # Calculate loss and correct predictions in batch
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        # Calculate positives and true positives in batch
        running_pos += labels.data.sum()
        running_pos_correct += ((preds == labels.data) & (labels.data == 1)).sum()
        
        # Calculate negatives and true negatives in batch
        running_neg += labels.data.sum()
        running_neg_correct += ((preds == labels.data) & (labels.data == 0)).sum()

    epoch_loss = running_loss / ds_size
    epoch_acc = running_corrects.double() / ds_size
    
    return epoch_loss, epoch_acc, (running_pos_correct, running_pos), (running_neg_correct, running_neg)
    
    

def eval_loop(model, dataloader, criterion, ds_size):
    '''Evaluate the model performance for one epoch'''

    # Put the model in evaluation mode to deactivate dropout
    model.eval()

    # Keep track of loss, predictions, and other numbers we are interested in
    # just like in the training loop
    running_loss = 0.0
    running_corrects = 0
    
    running_pos = 0
    running_pos_correct = 0
    running_neg = 0
    running_neg_correct = 0
    
    # Don't calculate gradients
    with torch.no_grad():
    
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
        
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
        
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            running_pos += labels.data.sum()
            running_pos_correct += ((preds == labels.data) & (labels.data == 1)).sum()

            running_neg += labels.data.sum()
            running_neg_correct += ((preds == labels.data) & (labels.data == 0)).sum()
        
    epoch_loss = running_loss / ds_size
    epoch_acc = running_corrects.double() / ds_size
    
    return epoch_loss, epoch_acc, (running_pos_correct, running_pos), (running_neg_correct, running_neg)

In [78]:
class LunaModel(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, padding=1, bias=True)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool3d(2)
        
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1, bias=True)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool3d(2)
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(2048, 1024)
        self.relu3 = nn.ReLU()
        
        self.dropout = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(1024, 2)
    
    def forward(self, X):
        
        # Dimensions of X => [BS, 1, 10, 18, 18]
        
        X = self.maxpool1(self.relu1(self.conv1(X)))
        X = self.maxpool2(self.relu2(self.conv2(X)))
        
        X = self.flatten(X)

        X = self.relu3(self.fc1(X))
        X = self.dropout(X)
        
        return self.fc2(X)
        

In [79]:
# Create an instance of the model
model = LunaModel()

# Use the GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Use the cross-entropy loss function
criterion = nn.CrossEntropyLoss()

# Use the AdamW optimizer
optimizer = optim.AdamW(model.parameters(), weight_decay=0.1)

In [82]:
EPOCHS = 100

for epoch in range(EPOCHS):

    epoch_start = time.time()

    train_loss, train_acc, train_pos, train_neg = train_loop(
        model, train_dl, criterion,
        optimizer, len(train_ds)
    )

    val_loss, val_acc, val_pos, val_neg = eval_loop(
        model, val_dl, criterion, len(val_ds)
    )

    time_elapsed = time.time() - epoch_start
    print(f'Epoch: {epoch+1:02} | Epoch Time: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print()
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tTrain - correct pos: {train_pos[0]}/{train_pos[1]} | correct neg: {train_neg[0]}/{train_neg[1]}')
    print()
    print(f'\tVal. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
    print(f'\tVal. - correct pos: {val_pos[0]}/{val_pos[1]} | correct neg: {val_neg[0]}/{val_neg[1]}')
    print()

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 5m 14s

	Train Loss: 11.142 | Train Acc: 57.35%
	Train - correct pos: 214/730 | correct neg: 629/730

	Val. Loss: 0.632 |  Val. Acc: 62.20%
	Val. - correct pos: 26/82 | correct neg: 76/82



  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch: 99 | Epoch Time: 0m 1s

	Train Loss: 0.244 | Train Acc: 91.50%
	Train - correct pos: 649/730 | correct neg: 696/730

	Val. Loss: 0.249 |  Val. Acc: 91.46%
	Val. - correct pos: 75/82 | correct neg: 75/82



  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch: 100 | Epoch Time: 0m 1s

	Train Loss: 0.192 | Train Acc: 93.67%
	Train - correct pos: 665/730 | correct neg: 712/730

	Val. Loss: 0.171 |  Val. Acc: 94.51%
	Val. - correct pos: 77/82 | correct neg: 78/82



In [88]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))


NameError: name 'y_true' is not defined